In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
from tqdm import tqdm

def scrape_object_details(object_url, driver):
    url = "https://tashkent.etagi.com" + object_url
    driver.get(url)
    time.sleep(2)  # Wait for the page to load fully

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    title = soup.select_one('[displayname="objectTitle"]').get_text(strip=True) if soup.select_one('[displayname="objectTitle"]') else "N/A"
    количество_комнат = re.search(r'(\d+)-комн', title)
    количество_комнат = количество_комнат.group(1) if количество_комнат else "N/A"
    тип = "квартира" if "квартира" in title.lower() else "студия" if "студия" in title.lower() else "N/A"
    
    валюта = "USD" if "у.е." in (цена_str := soup.select_one('.eypL8').get_text(strip=True) if soup.select_one('.eypL8') else "N/A").lower() else "UZS"
    цена = re.sub(r'[^\d]', '', цена_str)

    additional_details = {}
    ul_element = soup.select_one('ul.PpfZ1')
    if ul_element:
        for li in ul_element.find_all('li'):
            title = li.find('span', class_='Y65Dj').get_text(strip=True)
            value = li.find('span', class_='XVztD').get_text(strip=True)
            additional_details[title] = value

    дата_публикации_raw = soup.select_one('.o8Cyp span').get_text(strip=True) if soup.select_one('.o8Cyp span') else "N/A"
    дата_публикации_match = re.search(r'(\d{2}\.\d{2}\.\d{2})', дата_публикации_raw)
    дата_публикации = (дата_публикации_match.group(1)[:-2] + "20" + дата_публикации_match.group(1)[-2:]) if дата_публикации_match else "N/A"

    return {
        "Название": soup.select_one('.tv2WS').get_text(strip=True) if soup.select_one('.tv2WS') else "N/A",
        "Тип": тип,
        "Тип постройки": soup.select_one('.object-params__value--build-type').get_text(strip=True) if soup.select_one('.object-params__value--build-type') else "N/A",
        "Адрес": soup.select_one('.mfrBs').get_text(strip=True) if soup.select_one('.mfrBs') else "N/A",
        "Ремонт": additional_details.get('Ремонт', "N/A"),
        "Площадь": re.sub(r'[^\d]', '', additional_details.get('Общая площадь', "0 м²")),
        "Этаж": additional_details.get('Этаж/Этажность', "N/A").split()[0],
        "Этажность": additional_details.get('Этаж/Этажность', "N/A").split()[-1],
        "Количество комнат": количество_комнат,
        "Дата публикации": дата_публикации,
        "Валюта": валюта,
        "Цена": цена
    }

def scroll_down(driver, scroll_pause_time=1, max_scroll_attempts=2):
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_attempts = 0
    
    while scroll_attempts < max_scroll_attempts:
        # Scroll down to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait for new content to load
        time.sleep(scroll_pause_time)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # No more new content loaded
            break
        
        last_height = new_height
        scroll_attempts += 1

def get_object_hrefs(driver, page_url):
    driver.get(page_url)
    time.sleep(2)  # Initial wait for the page to load
    
    # Scroll to load all objects
    scroll_down(driver)

    # After scrolling, collect all the hrefs
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return [card['href'] for card in soup.select('a.templates-object-card__body.yQfYt[href]')]

def scrape_all_pages(base_url, max_pages, driver):
    data = []
    for page_num in tqdm(range(1, max_pages + 1), desc="Pages", unit="page"):
        page_url = f"{base_url}?page={page_num}"
        hrefs = get_object_hrefs(driver, page_url)
        if not hrefs:
            print(f"No more data found after page {page_num - 1}. Stopping.")
            break
        
        for href in tqdm(hrefs, desc="Objects", unit="object", leave=False):
            object_data = scrape_object_details(href, driver)
            if object_data:
                data.append(object_data)
            else:
                print(f"Failed to scrape data for {href}")
        
        time.sleep(2)
    
    return pd.DataFrame(data)

# Example usage
if __name__ == "__main__":
    driver = webdriver.Chrome()

    base_url = "https://tashkent.etagi.com/realty/?seller[]=owner"
    df = scrape_all_pages(base_url, max_pages=300, driver=driver)  # Example with max 10 pages
    driver.quit()

    df.shape

df['Тип постройки'] = 'вторичка'

Pages:  32%|███▏      | 97/300 [2:07:23<4:26:35, 78.80s/page]

No more data found after page 97. Stopping.


In [43]:
import pandas as pd
import string
import re

def clean_address(address):
    # Replace specific phrases
    address = address.replace('ул.', 'улица')
    address = address.replace('Ташкент', '')

    # Remove specific patterns (like '5 км до центра')
    address = re.sub(r'\d+\s*км\s*до\s*центра', '', address, flags=re.IGNORECASE)
    
    # List of district names in Tashkent
    districts = [
        'Бектемирский', 'Мирзо Улугбекский', 'Чилонзорский', 'Янгихаятский',
        'Шайхантахурский', 'Учтепинский', 'Яшнабадский', 'Мирабадский', 
        'Сергелийский', 'Юнусабадский', 'Чиланзарский', 'Алмазарский', 'Яшнободский'
    ]
    
    # Create a regex pattern to remove district names and the word 'район'
    pattern = r'\b(?:' + '|'.join(districts) + r')\s*район\b'
    
    # Remove the district names and 'район'
    address = re.sub(pattern, '', address, flags=re.IGNORECASE)
    
    # Remove punctuation
    address = address.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    
    # Remove extra spaces
    address = re.sub(r'\s+', ' ', address).strip()
    
    return address

df['Адрес'] = df['Адрес'].apply(clean_address)
df = df[~df['Адрес'].str.lower().str.contains('карте')]
df = df[~df['Адрес'].str.strip().eq('массив')]
df

,Название,Тип,Тип постройки,Адрес,Ремонт,Площадь,Этаж,Этажность,Количество комнат,Дата публикации,Валюта,Цена
0,Продается Евроквартира с готовым ремонтом в Жи...,квартира,вторичка,улица Сайрам,Евроремонт,42,7,9,1,08.10.2024,USD,95000
1,Продается уютная и светлая 3-комнатная квартир...,квартира,вторичка,улица Шахристанская,Косметический ремонт,69,3,4,3,08.10.2024,USD,91000
2,В продаже полноценная трехкомнатная квартира в...,квартира,вторичка,улица Миришкор,Улучшенная черновая отделка,9124,8,9,3,08.10.2024,USD,116000
3,Продается 2-комнатная квартира в Яшнабадском р...,квартира,вторичка,улица Фергана Йули,N/A,5638,6,9,2,08.10.2024,USD,57000
4,Продается великолепная 4-х комнатная квартира ...,квартира,вторичка,улица Султанали Машхади,Евроремонт,100,7,9,4,08.10.2024,USD,135000
...,...,...,...,...,...,...,...,...,...,...,...,...
2427,Продаётся уникальная квартира в самом сердце Т...,квартира,вторичка,массив Госпитальный,Частичный ремонт,70,3,5,2,12.08.2024,USD,125000
2428,Расположение дома вблизи от дороги и кафе дела...,квартира,вторичка,массив Караташ,Косметический ремонт,8269,2,5,3,06.06.2024,USD,100000
2429,Продается просторная квартира на массиве Башлы...,квартира,вторичка,Яккасарайский район массив Башлык,Косметический ремонт,97,8,9,4,N/A,USD,104000
2430,Продаётся просторная квартира с потрясающим по...,квартира,вторичка,улица проспект Мустакиллик,Косметический ремонт,94,3,9,4,05.08.2024,USD,135000


In [44]:
import pandas as pd
import time
import numpy as np
from tqdm import tqdm  # Import tqdm for the progress bar
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize WebDriver (open it once and reuse it)
def init_driver():
    driver = webdriver.Chrome()
    driver.get("https://www.google.com/maps")
    return driver

# Function to calculate cosine similarity
def calculate_cosine_similarity(input_address, suggestion_texts):
    vectorizer = TfidfVectorizer().fit_transform([input_address] + suggestion_texts)
    vectors = vectorizer.toarray()
    
    # Compute the cosine similarity between the input address and each suggestion
    input_vector = vectors[0]
    suggestion_vectors = vectors[1:]
    similarity_scores = cosine_similarity([input_vector], suggestion_vectors).flatten()
    
    return similarity_scores

# Function to find the most similar search suggestion
def select_most_similar_suggestion(driver, input_address):
    try:
        suggestions_elements = driver.find_elements(By.CSS_SELECTOR, 'div[jsaction="suggestion.select"]')
        suggestions_texts = [suggestion.text for suggestion in suggestions_elements]

        if not suggestions_texts:
            return False  # No suggestions available

        # Calculate cosine similarity between the input address and all suggestions
        similarity_scores = calculate_cosine_similarity(input_address, suggestions_texts)
        
        # Find the suggestion with the highest similarity
        best_match_index = np.argmax(similarity_scores)
        best_similarity_score = similarity_scores[best_match_index]

        # Only select if it's 60% or more similar
        if best_similarity_score >= 0.4:
            suggestions_elements[best_match_index].click()
            return True

        return False  # No suitable suggestion found
    except Exception as e:
        return False

def get_lat_long(driver, address):
    try:
        # Find the search box and input the address
        search_box = driver.find_element(By.NAME, "q")
        search_box.clear()  # Clear previous input before entering new address
        search_box.send_keys(address)
        search_box.submit()
        time.sleep(1)  # Wait for the search results to load

        # Select the most similar suggestion based on cosine similarity
        if not select_most_similar_suggestion(driver, address):
            return "N/A", "N/A"

        time.sleep(1)  # Wait for the page to load with the selected location

        # Extract latitude and longitude from the opened page itself
        url = driver.current_url

        # Parsing the latitude and longitude from the URL using 'll' or 'query'
        if "@" in url:
            coords_section = url.split("@")[1].split(",")  # The coordinates are usually after the '@' in the URL
            latitude = coords_section[0]
            longitude = coords_section[1]
            return latitude, longitude
        else:
            return "N/A", "N/A"
    except Exception as e:
        return "N/A", "N/A"

# Main function to apply the extraction to the DataFrame
def extract_lat_long_from_df(df, driver):
    latitudes = []
    longitudes = []

    # Use tqdm to create a progress bar
    for address in tqdm(df['Адрес'], desc="Extracting coordinates", unit="address"):
        latitude, longitude = get_lat_long(driver, address)
        latitudes.append(latitude)
        longitudes.append(longitude)
        time.sleep(1)  # Optional: delay between requests to avoid being blocked
    
    df['Широта'] = latitudes
    df['Долгота'] = longitudes
    return df

# Initialize the driver once
driver = init_driver()

# Extract latitude and longitude
df = extract_lat_long_from_df(df, driver)
driver.quit()

Extracting coordinates: 100%|██████████| 2432/2432 [1:36:22<00:00,  2.38s/address]


In [45]:
from datetime import datetime

columns_to_check = ["Источник", "Название", "Тип","Санузел", "Тип постройки", "Материал", "Широта", 
                    "Долгота", "Район", "Этаж", "Этажность", "Ремонт", "Площадь", 
                    "Количество комнат", "Дата публикации", "Валюта", "Цена", "Дата создания"]

# Create a new DataFrame with the specified columns
new_df = pd.DataFrame(columns=columns_to_check)

# Check if columns exist in final_df and create them with None values if not
for column in columns_to_check:
    if column not in df.columns:
        df[column] = None
        new_df[column] = None
    else:
        new_df[column] = df[column]

 
new_df["Источник"] = 'Etaji'
new_df["Район"] = ''
new_df["Дата создания"] = datetime.now().strftime("%d.%m.%Y")
new_df[columns_to_check]
new_df.head()

,Источник,Название,Тип,Санузел,Тип постройки,Материал,Широта,Долгота,Район,Этаж,Этажность,Ремонт,Площадь,Количество комнат,Дата публикации,Валюта,Цена,Дата создания
0,Etaji,Продается Евроквартира с готовым ремонтом в Жи...,квартира,None,вторичка,None,41.3317772,69.3218777,,7,9,Евроремонт,42,1,08.10.2024,USD,95000,09.10.2024
1,Etaji,Продается уютная и светлая 3-комнатная квартир...,квартира,None,вторичка,None,N/A,N/A,,3,4,Косметический ремонт,69,3,08.10.2024,USD,91000,09.10.2024
2,Etaji,В продаже полноценная трехкомнатная квартира в...,квартира,None,вторичка,None,41.3301202,69.3429507,,8,9,Улучшенная черновая отделка,9124,3,08.10.2024,USD,116000,09.10.2024
3,Etaji,Продается 2-комнатная квартира в Яшнабадском р...,квартира,None,вторичка,None,41.2671693,69.3114909,,6,9,N/A,5638,2,08.10.2024,USD,57000,09.10.2024
4,Etaji,Продается великолепная 4-х комнатная квартира ...,квартира,None,вторичка,None,41.3104098,69.3141232,,7,9,Евроремонт,100,4,08.10.2024,USD,135000,09.10.2024


In [46]:
# Specify columns to check for duplicates
columns_to_check_dup = ["Источник", "Название", "Тип", "Санузел", "Тип постройки", "Материал", 
                    "Широта", "Долгота", "Район", "Этаж", "Этажность", "Ремонт", "Площадь", 
                    "Количество комнат", "Дата публикации", "Валюта", "Цена"]

# Count the number of rows before removing duplicates
rows_before = new_df.shape[0]

# Remove duplicates based on specified columns
df_no_duplicates = new_df.drop_duplicates(subset=columns_to_check_dup, keep=False)

# Count the number of rows after removing duplicates
rows_after = df_no_duplicates.shape[0]

# Calculate the number of rows deleted
rows_deleted = rows_before - rows_after

print(f"\nNumber of rows deleted: {rows_deleted}")


Number of rows deleted: 526


In [47]:
import pandas as pd
import os

# Assuming x is your variable and data is the data you want to store
name_of_file = "Etaji"
df = pd.DataFrame(df_no_duplicates)

# Set the path to the Excels folder (assuming it is a sibling of the Notebooks folder)
excels_folder_path = r"D:\Documents\GitHub\data_scrapping\Data Scrapping\Excels"

# Check if the folder exists, if not, create it
if not os.path.exists(excels_folder_path):
    os.makedirs(excels_folder_path)

# Create a folder with the name_of_file only if it doesn't exist
file_folder_path = os.path.join(excels_folder_path, name_of_file)

if not os.path.exists(file_folder_path):
    os.makedirs(file_folder_path)

excel_file_name = os.path.join(file_folder_path, f"{name_of_file}.xlsx")

# Check if the file already exists
if os.path.exists(excel_file_name):
    # Read the existing Excel file into a DataFrame
    existing_df = pd.read_excel(excel_file_name)

    # Append the new data to the existing DataFrame
    updated_df = pd.concat([existing_df, df], ignore_index=True)

    # Check for duplicates in all columns
    duplicates_mask = updated_df.duplicated(keep=False)

    # Print the number of duplicates
    num_duplicates = duplicates_mask.sum()
    print(f"Number of duplicates after adding new data: {num_duplicates}")

    # If duplicates exist, remove them
    if any(duplicates_mask):
        updated_df = updated_df[~duplicates_mask]

    # Write the updated DataFrame back to the Excel file
    updated_df.to_excel(excel_file_name, index=False)

    print(f"Data added to existing Excel file '{excel_file_name}' after removing duplicates.")
else:
    # If the file doesn't exist, create a new Excel file with the data
    df.to_excel(excel_file_name, index=False)
    print(f"Excel file '{excel_file_name}' created with new data.")

Number of duplicates after adding new data: 0
Data added to existing Excel file 'D:\Documents\GitHub\data_scrapping\Data Scrapping\Excels\Etaji\Etaji.xlsx' after removing duplicates.
